In [ ]:
# Setup for collab, commment out otherwise
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# Change directory to the package folder
%cd '/content/drive/MyDrive/ElusiveImages'
# Verify the contents of the current folder
#!ls
# Install the requirements with pip
!pip install -r collab_requirements.txt

In [ ]:
import numpy as np
import torch

torch.cuda.empty_cache()
import torchvision.models as tvmodels
import torchvision.transforms as transforms
import yaml
from PIL import Image
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from database import BaseDatabase
from models import SiameseNet
from dataset import TripletData
from dataset import ContrastiveData
from dataset import QuadrupletData
from loss import TripletLoss
from loss import ContrastiveLoss
from loss import QuadrupletLoss
from MLPMixer.models.modeling import MlpMixer, CONFIGS
from torch import nn
import logging as log
from utils.MAP import compute_map
import matplotlib.pyplot as plt
from train import *

In [ ]:
# Seed value to reproduce results
seed_value = 123456
set_seed(seed_value)

all_params = {'batch_size': 256, 'learning_rate': 0.0001, 'reg': 0.05, 'epochs': 25, 'momentum': .9,
              'validevery': 100, 'train_path': 'dataset/flowers/train',
              'test_path': 'dataset/flowers/test',
              'val_path': 'dataset/flowers/test', 'optimizer': 'adam', 'data_size': 1000,
              'model': 'SiameseNet', 'category': 'resnet18', 'pretrain': False,
              'data': 'dataset/flowers', 'img_h': 384, 'img_w': 384, 'faiss_db': 'siamesedb',
              'save_db': True, 'loss_type': 'ContrastiveLoss', 'dataset': 'ContrastiveData',
              'loglevel': 'info', 'logdir': 'logs'}

log_file_name = setup_logging(all_params, seed_value)
run_experiment(all_params, log_file_name)

Setting seed value to 123456 for reproducing results
[INFO] RUNNING VALIDATION AT EPOCH 0
building database
Saving database to siamesedb
[INFO] CATEGORY MATCHES: 82/272: 0.3015
[INFO] Training Epoch 0


  0%|          | 0/5 [00:00<?, ?it/s]

[INFO] Validating Epoch 0


  0%|          | 0/2 [00:00<?, ?it/s]

[INFO] epoch 0: Loss = 98.17650604248047, Validation Loss = 4.712273597717285
[INFO] Training Epoch 1


...

  0%|          | 0/2 [00:00<?, ?it/s]

[INFO] epoch 1: Loss = 59.17056655883789, Validation Loss = 6.747849464416504
[INFO] Training Epoch 2


  0%|          | 0/5 [00:00<?, ?it/s]

[INFO] Validating Epoch 2


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
seed_value = 123456
set_seed(seed_value)

all_params = {'batch_size': 32, 'learning_rate': 0.0001, 'reg': 0.05, 'epochs': 25, 'momentum': .9,
              'validevery': 100, 'train_path': 'dataset/flowers/train',
              'test_path': 'dataset/flowers/test',
              'val_path': 'dataset/flowers/test', 'optimizer': 'adam', 'data_size': 1000,
              'model': 'SiameseNet', 'category': 'resnet18', 'pretrain': False,
              'data': 'dataset/flowers', 'img_h': 384, 'img_w': 384, 'faiss_db': 'siamesedb',
              'save_db': True, 'loss_type': 'TripletLoss', 'dataset': 'TripletData',
              'loglevel': 'info', 'logdir': 'logs'}

log_file_name = setup_logging(all_params, seed_value)
run_experiment(all_params, log_file_name)

In [ ]:
seed_value = 123456
set_seed(seed_value)

all_params = {'batch_size': 32, 'learning_rate': 0.0001, 'reg': 0.05, 'epochs': 25, 'momentum': .9,
              'validevery': 100, 'train_path': 'dataset/flowers/train',
              'test_path': 'dataset/flowers/test',
              'val_path': 'dataset/flowers/test', 'optimizer': 'adam', 'data_size': 1000,
              'model': 'SiameseNet', 'category': 'resnet18', 'pretrain': False,
              'data': 'dataset/flowers', 'img_h': 384, 'img_w': 384, 'faiss_db': 'siamesedb',
              'save_db': True, 'loss_type': 'QuadrupletLoss', 'dataset': 'QuadrupletData',
              'loglevel': 'info', 'logdir': 'logs'}

log_file_name = setup_logging(all_params, seed_value)
run_experiment(all_params, log_file_name)

Setting seed value to 123456 for reproducing results
[INFO] RUNNING VALIDATION AT EPOCH 0
building database
Saving database to siamesedb
[INFO] CATEGORY MATCHES: 96/272: 0.3529
[INFO] Training Epoch 0


  0%|          | 0/11 [00:00<?, ?it/s]

[INFO] Validating Epoch 0


  0%|          | 0/11 [00:00<?, ?it/s]

[INFO] epoch 0: Loss = 42.48470687866211, Validation Loss = 29.977588653564453
[INFO] Training Epoch 1


...

  0%|          | 0/11 [00:00<?, ?it/s]

[INFO] Validating Epoch 6


  0%|          | 0/11 [00:00<?, ?it/s]

Couldn't find new triplet
[INFO] epoch 6: Loss = 12.76438045501709, Validation Loss = 14.81568717956543
[INFO] Training Epoch 7


  0%|          | 0/11 [00:00<?, ?it/s]